In [1]:
import sys
import os
import cv2
import numpy as np
import random
from matplotlib import pyplot as plt
path_to_video = './Opt1-MarionetteMovements.mov'
%matplotlib inline

In [2]:
frame_save_path = './frames/'
background_save_path = './background/'
movement_save_path = './movement/'
point_save_path = './points/'

In [3]:
def create_dir_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [4]:
def calculate_SSD(input_img,x_pos,y_pos):
    # point is the left-right corner instead of center
    ssd = 0
    count = 0
    for y in range(y_pos-10,y_pos+11):
        for x in range(x_pos-10,x_pos+11):
            if(y >= input_img.shape[0] or y < 0 or x < 0 or x >= input_img.shape[1] or (input_img[y,x,0] == 255 and input_img[y,x,1] == 255 and input_img[y,x,2] == 255)):
                continue
            else:
                value = (int(input_img[y,x,2]) - int(input_img[y,x,1]) - int(input_img[y,x,0]))
                ssd += value
                count += 1
    if(count == 0):
        return -1
    return ssd/count

In [5]:
def find_points(input_img):
    # point is the left-right corner instead of center
    ssd = 0
    list_1 = []
    for y in range(0,input_img.shape[0]):
        for x in range(0,input_img.shape[1]):
            if input_img[y,x,2] == 0 or x%3 !=0 or y %3 != 0:
                pass
            else:
                ssd = calculate_SSD(input_img,x,y)
                if(ssd != -1):
                    list_1.append((y,x,ssd))
    list_2 = []
    max_ssd = list_1[0]
    while True:
        for i in list_1:
            if i[2] > max_ssd[2]:
                x = False
                for a in list_2:
                    if abs(a[0] - i[0]) <= 20 and abs(a[1] - i[1]) <= 20:
                        x = True
                if x:
                    continue
                else:
                    max_ssd = i
        list_2.append(max_ssd)
        max_ssd= list_1[0]
        if len(list_2) >= 5:
            break
    return list_2

# Retrieve the movement with black background

In [6]:
cap = cv2.VideoCapture(path_to_video)
BLUE = 60
create_dir_if_not_exists(movement_save_path) # Or you can create it manully
if not cap.isOpened():
    print('{} not opened'.format(path_to_video))
    sys.exit(1)
time_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frame_counter = 0 # FRAME_COUNTER
while(1):
    return_flag, monkeyframe = cap.read()  
    if not return_flag:
        print('Video Reach End')
        break
    # Main Content - Start
    for y in range(monkeyframe.shape[0]):
        for x in range(monkeyframe.shape[1]):
            if monkeyframe[y,x,0] >= BLUE and monkeyframe[y,x,1] < 180 and monkeyframe[y,x,2] < 180 and monkeyframe[y,x,0] > monkeyframe[y,x,1] and monkeyframe[y,x,0] > monkeyframe[y,x,2]:
                monkeyframe[y,x] = [0,0,0]
    cv2.imshow('monkey', monkeyframe)
    cv2.imwrite(movement_save_path + 'frame%d.tif' % frame_counter, monkeyframe)
    frame_counter += 1
    # Main Content - End    
    if cv2.waitKey(30) & 0xff == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

# Retrive the movemen feature and save as text file

In [7]:
frame_counter = 0
create_dir_if_not_exists(point_save_path)
with open("movement_feature.txt", 'w', encoding='utf-8') as file:
    while(1):
        img = cv2.imread(movement_save_path + 'frame%d.tif' % frame_counter)
        if img is None:
            print('No more frames to be loaded')
            break;
        points = find_points(img)
        file.write(str(frame_counter)+" ")
        print(str(frame_counter)+" ",end="")
        for point in points:
            y = point[0]
            x = point[1]
            file.write("(" + str(x) + "," + str(y) + ")")
            print("(" + str(x) + "," + str(y) + ")",end="")
        file.write("\n")
        print()
        frame_counter += 1


0 (264,168)(261,240)(285,78)(240,261)(183,129)


KeyboardInterrupt: 

# Show the effect of the movement detection algorithm

In [ ]:
frame_counter = 0
create_dir_if_not_exists(point_save_path)
while(1):
    img = cv2.imread(movement_save_path + 'frame%d.tif' % frame_counter)
    cv2.imshow('monkey', img)
    if img is None:
        print('No more frames to be loaded')
        break;
    points = find_points(img)
    for point in points:
        img[point[0],point[1],1] = 255
    cv2.imshow('monkey', img)
    cv2.imwrite(point_save_path + 'frame%d.tif' % frame_counter, img)
    frame_counter += 1
out.release()
cv2.destroyAllWindows()